In [1]:
import shutup
shutup.please()

import pandas as pd
import numpy as np
from datetime import datetime
import os, os.path, pickle, sys
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt

sys.path.append("/Users/hn/Documents/00_GitHub/Rangeland/Python_Codes/")
import rangeland_core as rc

In [2]:
data_dir_base = "/Users/hn/Documents/01_research_data/RangeLand/Data/"
census_population_dir = data_dir_base + "census/"
# Shannon_data_dir = data_dir_base + "Shannon_Data/"
# USDA_data_dir = data_dir_base + "/NASS_downloads/"
param_dir = data_dir_base + "parameters/"
Min_data_base = data_dir_base + "Min_Data/"
reOrganized_dir = data_dir_base + "reOrganized/"

plots_dir = data_dir_base + "plots/"

In [3]:
SoI = ["Alabama", "Arkansas", "California", 
       "Colorado", "Florida", "Georgia", 
       "Idaho", "Illinois", "Iowa", 
       "Kansas", "Kentucky", "Louisiana", 
       "Mississippi", "Missouri", "Montana", 
       "Nebraska", "New Mexico", "North Dakota", 
       "Oklahoma", "Oregon", "South Dakota", 
       "Tennessee", "Texas", "Virginia", 
       "Wyoming"]

abb_dict = pd.read_pickle(param_dir + "state_abbreviations.sav")
SoI_abb = []
for x in SoI:
    SoI_abb = SoI_abb + [abb_dict["full_2_abb"][x]]

In [4]:
USDA_data = pd.read_pickle(reOrganized_dir + "USDA_data.sav")
print ("-----------  reading USDA data  -----------")
cattle_inventory = USDA_data["cattle_inventory"]
cattle_inventory = cattle_inventory[["year", "cattle_cow_beef_inventory", "county_fips"]]
cattle_inventory.head(2)

-----------  reading USDA data  -----------


,year,cattle_cow_beef_inventory,county_fips
0,2017,8678.0,01001
1,2017,14589.0,01047


In [5]:
county_id_name_fips = pd.read_csv(Min_data_base + "county_id_name_fips.csv")
county_id_name_fips.rename(columns=lambda x: x.lower().replace(" ", "_"), inplace=True)

county_id_name_fips.sort_values(by=["state", "county"], inplace=True)
county_id_name_fips.rename(columns={"county": "county_fips"}, inplace=True)

county_id_name_fips = rc.correct_Mins_FIPS(df=county_id_name_fips, col_="county_fips")
county_id_name_fips.reset_index(drop=True, inplace=True)
county_id_name_fips.head(2)

,county_fips,county_name,fips,state
0,02013,Aleutians East Borough,2013,AK
1,02016,Aleutians West Census Area,2016,AK


In [6]:
cattle_inventory_2002 = cattle_inventory[cattle_inventory.year == 2002].copy()
cattle_inventory_2017 = cattle_inventory[cattle_inventory.year == 2017].copy()

cattle_inventory_2002.reset_index(drop=True, inplace=True)
cattle_inventory_2017.reset_index(drop=True, inplace=True)

inv_col = "cattle_cow_beef_inventory"
cattle_inventory_2002.rename(columns={inv_col: "cattle_cow_beef_inventory_2002"}, inplace=True)
cattle_inventory_2017.rename(columns={inv_col: "cattle_cow_beef_inventory_2017"}, inplace=True)
cattle_inventory_2002.head(2)

,year,cattle_cow_beef_inventory_2002,county_fips
4553,2002,6547.0,01011
4554,2002,8030.0,01051


In [7]:
invent_2002_2017 = pd.merge(cattle_inventory_2002[["cattle_cow_beef_inventory_2002", "county_fips"]], 
                            cattle_inventory_2017[["cattle_cow_beef_inventory_2017", "county_fips"]], 
                            on = ["county_fips"], how = "left")

invent_2002_2017.head(2)

,cattle_cow_beef_inventory_2002,county_fips,cattle_cow_beef_inventory_2017
0,6547.0,01011,NaN
1,8030.0,01051,6893.0


In [8]:
print (invent_2002_2017.shape)
invent_2002_2017.dropna(how='any', inplace=True)
print (invent_2002_2017.shape)

(1627, 3)
(1245, 3)


In [9]:
invent_2002_2017.head(2)

,cattle_cow_beef_inventory_2002,county_fips,cattle_cow_beef_inventory_2017
1,8030.0,01051,6893.0
3,21826.0,01085,17865.0


In [10]:
invent_2002_2017["cattle_cow_beef_inv_change2002to2017"] = invent_2002_2017["cattle_cow_beef_inventory_2017"] - \
                                                           invent_2002_2017["cattle_cow_beef_inventory_2002"]
    
invent_2002_2017.head(2)

,cattle_cow_beef_inventory_2002,county_fips,cattle_cow_beef_inventory_2017,cattle_cow_beef_inv_change2002to2017
1,8030.0,01051,6893.0,-1137.0
3,21826.0,01085,17865.0,-3961.0


In [11]:
invent_2002_2017 = invent_2002_2017[["county_fips", "cattle_cow_beef_inv_change2002to2017"]]
invent_2002_2017.reset_index(drop=True, inplace=True)
invent_2002_2017.head(2)

,county_fips,cattle_cow_beef_inv_change2002to2017
0,01051,-1137.0
1,01085,-3961.0


In [12]:
out_name = data_dir_base + "data_4_plot/" + "inventory_AbsChange_2002to2017.csv"
invent_2002_2017.to_csv(out_name, index=False)

In [13]:
cattle_inventory.head(2)

,year,cattle_cow_beef_inventory,county_fips
0,2017,8678.0,01001
1,2017,14589.0,01047


In [14]:
county_id_name_fips.head(2)

,county_fips,county_name,fips,state
0,02013,Aleutians East Borough,2013,AK
1,02016,Aleutians West Census Area,2016,AK


In [15]:
cattle_inventory = pd.merge(cattle_inventory, county_id_name_fips, on = ["county_fips"], how = "left")
cattle_inventory.head(2)

,year,cattle_cow_beef_inventory,county_fips,county_name,fips,state
0,2017,8678.0,01001,Autauga County,1001.0,AL
1,2017,14589.0,01047,Dallas County,1047.0,AL


In [16]:
len(cattle_inventory.state.unique())

26

In [17]:
cattle_inventory[~(cattle_inventory.state.isin(SoI_abb))]

,year,cattle_cow_beef_inventory,county_fips,county_name,fips,state
2686,2012,20938.0,46102,NaN,NaN,NaN


In [18]:
county_id_name_fips[county_id_name_fips.county_fips == "46102"]

,county_fips,county_name,fips,state


In [19]:
cattle_inventory.dropna(how='any', inplace=True)
len(cattle_inventory.state.unique())

25

# Compute as percentage

In [21]:
cattle_inventory_2002 = cattle_inventory[cattle_inventory.year == 2002].copy()
cattle_inventory_2017 = cattle_inventory[cattle_inventory.year == 2017].copy()

cattle_inventory_2002.reset_index(drop=True, inplace=True)
cattle_inventory_2017.reset_index(drop=True, inplace=True)


inv_col = "cattle_cow_beef_inventory"
cattle_inventory_2002.rename(columns={inv_col: "cattle_cow_beef_inventory_2002"}, inplace=True)
cattle_inventory_2017.rename(columns={inv_col: "cattle_cow_beef_inventory_2017"}, inplace=True)
cattle_inventory_2002.head(2)

,year,cattle_cow_beef_inventory_2002,county_fips,county_name,fips,state
0,2002,6547.0,01011,Bullock County,1011.0,AL
1,2002,8030.0,01051,Elmore County,1051.0,AL


In [22]:
cattle_inventory_2002_total = cattle_inventory_2002.cattle_cow_beef_inventory_2002.sum()
cattle_inventory_2017_total = cattle_inventory_2017.cattle_cow_beef_inventory_2017.sum()

In [39]:
cattle_inventory_2002["cattle_cow_beef_inv_2002_asPerc"] = 100*\
                          (cattle_inventory_2002["cattle_cow_beef_inventory_2002"]/cattle_inventory_2002_total)

cattle_inventory_2017["cattle_cow_beef_inv_2017_asPerc"] = 100*\
                          (cattle_inventory_2017["cattle_cow_beef_inventory_2017"]/cattle_inventory_2017_total)

cattle_inventory_2017.drop(columns=['fips'], inplace=True)
cattle_inventory_2002.drop(columns=['fips'], inplace=True)

cattle_inventory_2002.head(2)

,year,cattle_cow_beef_inventory_2002,county_fips,county_name,state,cattle_cow_beef_inv_2002_asPerc
0,2002,6547.0,01011,Bullock County,AL,0.026107
1,2002,8030.0,01051,Elmore County,AL,0.032021


In [40]:
cattle_inventory_2017.head(2)

,year,cattle_cow_beef_inventory_2017,county_fips,county_name,state,cattle_cow_beef_inv_2017_asPerc
0,2017,8678.0,01001,Autauga County,AL,0.040422
1,2017,14589.0,01047,Dallas County,AL,0.067955


In [42]:
invent_2002_2017_asPerc = pd.merge(cattle_inventory_2002[["cattle_cow_beef_inv_2002_asPerc", "county_fips"]], 
                                   cattle_inventory_2017[["cattle_cow_beef_inv_2017_asPerc", "county_fips"]], 
                                   on = ["county_fips"], how = "left")

invent_2002_2017_asPerc.head(2)

,cattle_cow_beef_inv_2002_asPerc,county_fips,cattle_cow_beef_inv_2017_asPerc
0,0.026107,01011,NaN
1,0.032021,01051,0.032107


In [44]:
print (invent_2002_2017_asPerc.shape)
invent_2002_2017_asPerc.dropna(how='any', inplace=True)
print (invent_2002_2017_asPerc.shape)

(1627, 3)
(1245, 3)


In [45]:
invent_2002_2017_asPerc.head(2)

,cattle_cow_beef_inv_2002_asPerc,county_fips,cattle_cow_beef_inv_2017_asPerc
1,0.032021,01051,0.032107
3,0.087036,01085,0.083215


In [48]:
invent_2002_2017_asPerc["change_2002_2017_asPerc"] = invent_2002_2017_asPerc["cattle_cow_beef_inv_2017_asPerc"] - \
                                                     invent_2002_2017_asPerc["cattle_cow_beef_inv_2002_asPerc"]
invent_2002_2017_asPerc.head(2)

,cattle_cow_beef_inv_2002_asPerc,county_fips,cattle_cow_beef_inv_2017_asPerc,change_2002_2017_asPerc
1,0.032021,01051,0.032107,0.000086
3,0.087036,01085,0.083215,-0.003821


In [49]:
out_name = data_dir_base + "data_4_plot/" + "inventory_PercChange_2002_2017.csv"
invent_2002_2017_asPerc[["county_fips", "change_2002_2017_asPerc"]].to_csv(out_name, index=False)

In [50]:
data_dir_base

'/Users/hn/Documents/01_research_data/RangeLand/Data/'

In [67]:
bads = ['Alaska',
        'American Samoa',
        'Guam',
        'Northern Mariana Islands',
        'Puerto Rico',
        'United States Minor Outlying Islands',
        'U.S. Virgin Islands']

all_abbr_ = []
for x in abb_dict["full_2_abb"].keys():
    if not(x in bads):
        all_abbr_ = all_abbr_ + [abb_dict["full_2_abb"][x]]
    
print (all_abbr_)

['AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'DC']


In [76]:
invent_2002_2017_asPerc[invent_2002_2017_asPerc.change_2002_2017_asPerc>0.2]

,cattle_cow_beef_inv_2002_asPerc,county_fips,cattle_cow_beef_inv_2017_asPerc,change_2002_2017_asPerc
123,0.124301,06107,0.338998,0.214697
